In [1]:
from lmfdb import db
from lmfdb_tools import ec_growth_torsion

INFO:LMFDB@2024-05-11 16:22:15,759: Configuration = {'flask_options': {'port': 37777, 'host': '127.0.0.1', 'debug': True, 'use_reloader': False}, 'postgresql_options': {'port': 5432, 'host': 'devmirror.lmfdb.xyz', 'dbname': 'lmfdb', 'user': 'lmfdb', 'password': '****'}, 'logging_options': {'logfile': 'flasklog', 'slowcutoff': 0.1, 'slowlogfile': 'slow_queries.log', 'editor': '', 'loglevel': 20}} 
INFO:LMFDB@2024-05-11 16:22:15,888: Connecting to PostgresSQL server as: user=lmfdb host=devmirror.lmfdb.xyz port=5432 dbname=lmfdb... 
INFO:LMFDB@2024-05-11 16:22:16,950: Done!
 connection = <connection object at 0x73e18f14e200; dsn: 'user=lmfdb password=xxx dbname=lmfdb host=devmirror.lmfdb.xyz port=5432', closed: 0> 
2024-05-11 16:22:17,869 - SELECT pg_is_in_recovery() ran in  0.12526774406433105s 
2024-05-11 16:22:18,358 - SELECT current_setting('is_superuser') ran in  0.3618767261505127s 
INFO:LMFDB@2024-05-11 16:22:18,560: User: lmfdb 
INFO:LMFDB@2024-05-11 16:22:18,561: Read only: True 

In [2]:
table = db.ec_torsion_growth.join_search(query={'degree' : int(2)}, 
                                        projection=["lmfdb_label", "torsion", "degree", "field", "conductor", ("ec_curvedata", "torsion_structure")], 
                                        join=[("ec_torsion_growth", "lmfdb_label", "ec_curvedata","lmfdb_label")])

ecs = list(table)
len(ecs)

2024-05-11 16:22:21,167 - SELECT "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."torsion", "ec_torsion_growth"."degree", "ec_torsion_growth"."field", "ec_torsion_growth"."conductor", "ec_curvedata"."torsion_structure" FROM "ec_torsion_growth" JOIN "ec_curvedata" ON "ec_torsion_growth"."lmfdb_label" = "ec_curvedata"."lmfdb_label" WHERE "ec_torsion_growth"."degree" = 2 ORDER BY "ec_torsion_growth"."lmfdb_label", "ec_torsion_growth"."degree" ran in  0.33416223526000977s 
2024-05-11 16:22:21,169 - Replicate with db.ec_torsion_growth.analyze({'degree': 2}, ['lmfdb_label', 'torsion', 'degree', 'field', 'conductor', ('ec_curvedata', 'torsion_structure')], None, 0)
2024-05-11 16:25:15,010 - Search iterator for ec_torsion_growth {'degree': 2} required a total of 168.2619435787201s


1661447

In [3]:
x = polygen(ZZ,'x')

for ec in ecs:
    ec['torsion_before'] = ec[('ec_curvedata','torsion_structure')]
    ec['torsion_after'] = ec['torsion']
    
    del ec['torsion']
    del ec[('ec_curvedata','torsion_structure')]
    
    
    f = ec['field']
    K.<alpha> = NumberField(f[0]*x^2+ f[1]*x + f[2]) 
    ec['disc_abs'] = K.absolute_discriminant()

In [5]:
ecs[0]

{'lmfdb_label': '100.a1',
 'degree': 2,
 'field': [-1, -1, 1],
 'conductor': 100,
 'torsion_before': [2],
 'torsion_after': [2, 2],
 'disc_abs': 5}

In [23]:
def prod(xs):
    prod = 1
    for x in xs:
        prod *= x
    return prod

def conditions(curve,  dict_key_property):
    boolean = True
    for key, propertyy in  dict_key_property.items():
        boolean = boolean and propertyy(curve[key])
    return boolean

def growth_torsion_search(dict_possible_torsion_growth, curves_list):
    return [ec for ec in curves_list if (tuple(ec['torsion_before']) in dict_possible_torsion_growth.keys()) 
            and (ec['torsion_after'] in dict_possible_torsion_growth[tuple(ec['torsion_before'])])]
    
    

In [24]:
dict_conditions = {'conductor' : lambda cond : cond%3 != 0, 
                   'torsion_before' : lambda tor_list : prod(tor_list)% 3 != 0,
                   'torsion_after' : lambda tor_list: prod(tor_list)%3 == 0,
                   'disc_abs' : lambda disc: disc%3 == 0}

ecs_trivial2C3 = [ec for ec in ecs if  conditions(ec, dict_conditions)]

In [25]:
counterexamples = [ec for ec in ecs_trivial2C3 if prod(ec['torsion_after'])/prod(ec['torsion_before']) != 3]
len(counterexamples)

0

In [26]:
len(ecs_trivial2C3)

35163

In [44]:
dict_search= {() : [5], (2,): [[10],[2,10]],(3,) : [15]}

ecs_5_growth = growth_torsion_search(dict_search, ecs)

In [45]:
len(ecs_5_growth)

530

In [50]:
ecs_5_growth[-1]

{'lmfdb_label': '98154.bl3',
 'degree': 2,
 'field': [1, -1, 1],
 'conductor': 98154,
 'torsion_before': [2],
 'torsion_after': [10],
 'disc_abs': -3}

In [51]:
factor(98154)

2 * 3^2 * 7 * 19 * 41

In [49]:
factor(56)

2^3 * 7

In [6]:
db.ec_nfcurves.join_search({"rank":1}, ["label", ("nf_fields", "r2")], [("ec_nfcurves", "field_label", "nf_fields", "label")], limit=3)


2024-05-10 17:20:08,808 - SELECT count FROM "ec_nfcurves_counts" WHERE cols = '["rank"]' AND values = '[1]' AND split = false ran in  1.1579384803771973s 
2024-05-10 17:20:12,528 - SELECT "ec_nfcurves"."label", "nf_fields"."r2" FROM "ec_nfcurves" JOIN "nf_fields" ON "ec_nfcurves"."field_label" = "nf_fields"."label" WHERE "ec_nfcurves"."rank" = 1 ORDER BY "ec_nfcurves"."field_label", "ec_nfcurves"."conductor_norm", "ec_nfcurves"."conductor_label", "ec_nfcurves"."iso_nlabel", "ec_nfcurves"."number" LIMIT 1000 ran in  3.5903468132019043s 
2024-05-10 17:20:12,529 - Replicate with db.ec_nfcurves.analyze({'rank': 1}, ['label', ('nf_fields', 'r2')], 1000, 0)


[{'label': '2.0.11.1-47.1-a1', ('nf_fields', 'r2'): 1},
 {'label': '2.0.11.1-47.2-a1', ('nf_fields', 'r2'): 1},
 {'label': '2.0.11.1-108.1-a1', ('nf_fields', 'r2'): 1}]